<h3> Machine Learning Project 6 </h3>
    <p> Liora Wachsstock </p>
    

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
#constants
UPPER = 10
LOWER = -2

In [19]:
def generate_data(n):
    rng = np.random.default_rng()
    x = (UPPER - LOWER) * rng.random(n) + LOWER
    y = f(x)
    data = np.stack((x, y), axis=-1)
    
    return pd.DataFrame(data, columns=("X", "Y"))

def f(x):
    return np.square(x) + 10

#generate data
train = generate_data(12)
test = generate_data(5)

#check for intersecting values and re-sample if necessary
while (np.intersect1d(train.T[0], test.T[0]).size > 0): #only have to check if there are identical values in x values
    print(np.intersect1d(train.T[0], test.T[0]))
    test = generate_data(5)

print(test)
print(train)

          X          Y
0  5.920119  45.047813
1  7.607975  67.881290
2  7.708739  69.424664
3  1.316587  11.733403
4  9.376952  97.927233
           X          Y
0   8.712675  85.910700
1   0.326665  10.106710
2  -1.693042  12.866391
3   2.204525  14.859930
4   4.587867  31.048527
5   9.091958  92.663705
6   9.134704  93.442817
7   4.677705  31.880924
8   3.042423  19.256340
9   0.797647  10.636241
10  3.980229  25.842220
11  6.096005  47.161271
